## Load in Packages and Files

In [1]:
using CSV
using DataFrames
using Random
using JuMP, Dates
using Gurobi
using StatsBase
using Clustering
using Statistics
using LinearAlgebra

# Define file paths
node_crime_counts_path = "node_crime_counts.csv"
distance_matrix_path = "distance_matrix_1880.csv"
subgraph_nodes = CSV.read("down_sampled_node_data_900.csv", DataFrame)
candidate_nodes = CSV.read( "candidate_nodes_900_60.csv", DataFrame)
subgraph_node_ids = Set(subgraph_nodes.node_id)
candidate_node_ids = Set(candidate_nodes.node_id)

# Load the files
cambridge_crime_path = "cambridge_crime_sizes_900.csv"
crime_category_path = "crime_categories.csv"
cambridge_crime = CSV.read(cambridge_crime_path, DataFrame)
crime_category = CSV.read(crime_category_path, DataFrame)
crime_to_category = Dict(row["Crime Type"] => row["Category_Num"] for row in eachrow(crime_category))

#Set Random Seed
Random.seed!(42)

node_crime_counts = CSV.read(node_crime_counts_path, DataFrame)
cambridge_crime.c = [crime_to_category[crime] for crime in cambridge_crime.Crime]

# For the distance matrix with the first column as the index
distance_matrix = CSV.read(distance_matrix_path, DataFrame)
index_lookup = Dict(value => i for (i, value) in enumerate(distance_matrix[:, 1]))

# Create a column lookup dictionary: column names to column indices
column_lookup = Dict(name => i for (i, name) in enumerate(names(distance_matrix)));

#row_id = index_lookup[61323200]  # This will give you the row index
#distance_matrix;
#distance_matrix[index_lookup[61321144], string(61182955)]

## Preprocessing Data

In [2]:
# Parse the "Date of Report" column as DateTime
cambridge_crime."Date of Report" = DateTime.(cambridge_crime."Date of Report", dateformat"yyyy-mm-dd HH:MM:SS")

# Find the earliest time in the "Date of Report" column
start_time = minimum(cambridge_crime."Date of Report")

# Calculate the hours since the earliest time and create a new column
cambridge_crime."hour_index" = round.(Int, (cambridge_crime."Date of Report" .- start_time) ./ Hour(1))

# Display the first few rows to check the result
#first(cambridge_crime, 5)

crime_2022_2023 = filter(row -> year(row."Date of Report") in 2022:2023, cambridge_crime) # Filter rows for 2022-2023
crime_2024 = filter(row -> year(row."Date of Report") == 2024, cambridge_crime); # Filter rows for 2024

## Create Scenarios

In [3]:
# Define function to assign 2 full year periods (2022 and 2023)
function assign_scenario(dt::DateTime)::Int
    if year(dt) == 2022 && month(dt) in 1:6
        return 1  #Jan-Jun 2022
    elseif year(dt) == 2022 && month(dt) in 7:12
        return 1  # Jul-Dec 2022
    elseif year(dt) == 2023 && month(dt) in 1:6
        return 2  # Jan-Jun 2023
    elseif year(dt) == 2023 && month(dt) in 7:12
        return 2  # Jul-Dec 2023
    else
        return 0  # Not in 2022-2023
    end
end

# Assign scenarios to the filtered data
crime_2022_2023."scenario" = assign_scenario.(crime_2022_2023."Date of Report");

## More Data Preparations For Optimization 

In [4]:
index_lookup = Dict(value => i for (i, value) in enumerate(distance_matrix[:, 1]))
column_lookup = Dict(name => i for (i, name) in enumerate(names(distance_matrix)))

# --- Prepare Index Mappings ---

# Nodes
node_ids = unique(cambridge_crime[!,"900_closest_node"]) # Change here
num_nodes = length(node_ids)
node_id_to_idx = Dict(node_ids[i] => i for i in 1:num_nodes)
node_idx_to_id = Dict(i => node_ids[i] for i in 1:num_nodes)
nodeset = 1:num_nodes

# Crime Types
crime_types = unique(cambridge_crime.c)
num_crime_types = length(crime_types)
crime_type_id_to_idx = Dict(crime_types[i] => i for i in 1:num_crime_types)
crime_type_idx_to_id = Dict(i => crime_types[i] for i in 1:num_crime_types)
crime_typeset = 1:num_crime_types

# Time Periods
time_periods = unique(cambridge_crime.hour_index)
time_periods = sort(time_periods)
num_time_periods = length(time_periods)
hour_to_time_idx = Dict(time_periods[i] => i for i in 1:num_time_periods)
time_idx_to_hour = Dict(i => time_periods[i] for i in 1:num_time_periods)
timeset = 1:num_time_periods

# --- Build N_ict ---
N_ic = zeros(Int, num_nodes, num_crime_types)  # Use `Int` for count storage
for row in eachrow(crime_2022_2023)
    node_id = row["900_closest_node"]  # Adjust column name as necessary
    crime_type = row["c"]          # Adjust column name as necessary
    if haskey(node_id_to_idx, node_id) && haskey(crime_type_id_to_idx, crime_type)
        i = node_id_to_idx[node_id]         # Get the row index for the node
        c = crime_type_id_to_idx[crime_type]  # Get the column index for the crime type
        N_ic[i, c] += 1                     # Increment the count in the matrix
    end
end

# Build N_ics for 2022-2023
N_ics = [zeros(Int, num_nodes, num_crime_types) for _ in 1:2]#4]

for row in eachrow(crime_2022_2023)
    node_id = row["900_closest_node"]
    crime_type = row["c"]
    scenario = row["scenario"]
    if haskey(node_id_to_idx, node_id) && haskey(crime_type_id_to_idx, crime_type) && scenario > 0
        i = node_id_to_idx[node_id]
        c = crime_type_id_to_idx[crime_type]
        N_ics[scenario][i, c] += 1
    end
end

# Build N_ic2024
N_ic2024 = zeros(Int, num_nodes, num_crime_types)
for row in eachrow(crime_2024)
    node_id = row["900_closest_node"]
    crime_type = row["c"]
    if haskey(node_id_to_idx, node_id) && haskey(crime_type_id_to_idx, crime_type)
        i = node_id_to_idx[node_id]
        c = crime_type_id_to_idx[crime_type]
        N_ic2024[i, c] += 1
    end
end

# --- Build d_ij ---
M = 1e6  # A large number
d_ij = M * ones(num_nodes, num_nodes)
for i in 1:num_nodes
    node_id_i = node_idx_to_id[i]
    if haskey(index_lookup, node_id_i)
        row_index = index_lookup[node_id_i]
        for j in 1:num_nodes
            node_id_j = node_idx_to_id[j]
            column_name = string(node_id_j)
            if haskey(column_lookup, column_name)
                distance = distance_matrix[row_index, column_name]
                if !ismissing(distance)
                    d_ij[i, j] = distance
                end
            end
        end
    end
end

for i in 1:size(d_ij, 1)
    d_ij[i, i] = 0
end

# --- Parameters ---
# Synthetic parameter values
t_cost = 0.1  # Transportation cost per meter

# Penalties
θ_c = Dict(c => 50 * c for c in crime_types)  # Penalty for not attending at all
π_c = Dict(1 => 500, 2 => 10000, 3 => 150000)  # Penalty for not attending quickly
r_c = Dict(1 => 2, 2 => 4, 3 => 8)     

# Total officers and maximum stations
total_officers = 281
max_stations = 1

1

## Model Formulation

First Stage:

  $x_j$: Binary variable indicating if a police station is built at node $j$

  $y_{js}$: Integer variable representing the number of officers assigned to station $j$


Second Stage:
  
$z_{jics}: $ # of officers from station j to node i for crime type c

$u_{ics}$: # of late responses at node i of crime c at time t

$z'_{jics}: $ # of officers from station j to node i for crime type c (response not quick enough)




Static parameters:

$d_{ij}$ is the distance from node i to node j 

$N_{ics}: $ # of crime c at node i

$r_{c}:$ # police requirement for crime type c

$\pi_c$ is the penalty for not attending to a crime of type c quick enough

$t$ is transportation cost



Optimization Formulation (simple):


\begin{align*}
\text{Minimize } & \quad \sum_{i,c} \pi_c \, u_{\text{ic}} + \sum_{i,j} d_{ij} \, t \left(\frac{z_{ijc} + z'_{jic}}{2}\right) \\
\text{subject to } & \quad \sum_j x_j = 3 \quad \text{(Facility Constraint)} \\
& \quad \sum_j y_j = 281 \quad \text{(Police Officer Constraint)} \\
& \quad y_j \leq M x_j, \quad \forall j \quad \text{(Don't Assign)} \\
& \quad y_j \geq 40 - M(1-x_j) \quad \forall j  \quad \text{(At least 40 officers)} \\
& \quad \sum_{j} (z_{jic}) + r_c u_{ic} = r_c N_{ic} \quad \text{(Now recording the number of officer visited from node j crime type c at node i throughout time)} \\
& \quad a_{ij} \in \{0, 1\} \\
& \quad s_{j} \in \{0, 1\} \\
& \quad D_j = 1000 + 1000s_j \quad \forall j \quad \text{(Radius Distance that can covered by station j. s\_j = 1 -> D\_j = 2000 else D\_j = 1000)} \\
& \quad \sum_{j}s_{j} = 1  \quad \text{(Can choose at most 1 station to have larger Radius distance)} \\
& \quad y_j \geq 100s_j \quad \forall j \quad \text{(s\_j = 1 -> y\_j >= 100)} \\
& \quad y_j \leq 99(1-s_j) + Ms_j\quad \forall j \quad \text{(s\_j = 1 -> y\_j >= 100)} \\
& \quad y_j \geq 0 \quad \forall j \\
& \quad (d_{ij} - D_j)/M \leq a_{ij} \forall j, i \\
& \quad z_{jic} \leq M (1 - a_{ij}) \quad \forall i,j,c \\
& \quad z'_{jic} \leq M a_{ij} \quad \forall i,j,c  \\
& \quad \sum_{j} z'_{jic}  = r_c u_{ic} \quad \forall i,c  \\
& \quad \sum_{i,c} z_{jic} + \sum_{i,c} z'_{jic} \leq Mx_j \quad \forall j \\
& \quad z_{jic} \geq 0 \quad \forall i,j,c \\
& \quad z'_{jic} \geq 0 \quad \forall i,j,c \\
\end{align*}




## Deterministic Optimization (stage 1)

In [5]:
# Initialize model
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids if haskey(node_id_to_idx, node))
model_ev_1 = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  #Limit crime types

# Variables
@variable(model_ev_1, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_ev_1, y[j in limited_candidateset] >= 0)  # Integer: Officers assigned to station j
@variable(model_ev_1, s[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_ev_1, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_ev_1, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_ev_1, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_ev_1, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_ev_1, D[j in limited_candidateset], 1000 + 1000 * s[j])  # Coverage radius

# Objective Function
@objective(model_ev_1, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_ev_1, sum(x[j] for j in limited_candidateset) == 3)  # Facility constraint: exactly 3 stations
@constraint(model_ev_1, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers
@constraint(model_ev_1, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_ev_1, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_ev_1, [j in limited_candidateset], y[j] >= 100 * s[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_ev_1, [j in limited_candidateset], y[j] <= 99 * (1 - s[j]) + 300 * s[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_ev_1, sum(s[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_ev_1, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_ev_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_ev_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_ev_1, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_ev_1, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_ev_1, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_ev_1, [j in limited_candidateset], y[j] >= 0)
@constraint(model_ev_1, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_ev_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_ev_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_ev_1, "TimeLimit", 300)  # Time limit for solving
set_optimizer_attribute(model_ev_1, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_ev_1, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_ev_1)

# Results
if termination_status(model_ev_1) == MOI.OPTIMAL || termination_status(model_ev_1) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("Deterministic Optimization 2022-23 (first stage):")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_ev_1))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 710463 rows, 380880 columns and 1736880 nonzeros
Model fingerprint: 0x4f506c7d
Variable types: 326760 continuous, 54120 integer (54120 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [5e-01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-09, 4e+04]
Presolve removed 628589 rows and 217478 columns
Presolve time: 0.30s
Presolved: 81874 rows, 163402 columns, 454318 nonzeros
Variable types: 149060 continuous, 14342 integer (14342 bi

In [6]:
# Get Stage 1 Variable Optimal Values
optimal_x_vals_model_ev_1 = value.(x)
optimal_y_vals_model_ev_1 = value.(y)
optimal_s_vals_model_ev_1 = value.(s)
optimal_a_vals_model_ev_1 = value.(a);

## Deterministic Optimization (stage 2)

In [7]:
# Initialize model
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids if haskey(node_id_to_idx, node))
model_ev_2 = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2#4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 2 Variables
@variable(model_ev_2, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_ev_2, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_ev_2, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_ev_2, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_model_ev_1[j])  # Coverage radius

# Objective Function
@objective(model_ev_2, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_ev_2, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] <= 300 * optimal_x_vals_model_ev_1[j])  # Don't assign officers unless station exists
@constraint(model_ev_2, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] >= 40 - 300 * (1 - optimal_x_vals_model_ev_1[j]))  # At least 40 officers if station exists
@constraint(model_ev_2, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] >= 100 * optimal_s_vals_model_ev_1[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_ev_2, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] <= 99 * (1 - optimal_s_vals_model_ev_1[j]) + 300 * optimal_s_vals_model_ev_1[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_ev_2, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_model_ev_1[j, i]
)

# Quick response constraint
@constraint(model_ev_2, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - optimal_a_vals_model_ev_1[j, i])
)

# Late response constraint
@constraint(model_ev_2, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * optimal_a_vals_model_ev_1[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_ev_2, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_ev_2, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_ev_2, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_model_ev_1[j]
)

# Non-negativity constraints
@constraint(model_ev_2, [j in limited_candidateset], optimal_x_vals_model_ev_1[j] >= 0)
@constraint(model_ev_2, [j in limited_candidateset], optimal_s_vals_model_ev_1[j] >= 0)
@constraint(model_ev_2, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_ev_2, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_ev_2, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_ev_2, "TimeLimit", 600)  # Increased time limit for stochastic optimization
set_optimizer_attribute(model_ev_2, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_ev_2, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_ev_2)

# Results
if termination_status(model_ev_2) == MOI.OPTIMAL || termination_status(model_ev_2) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(optimal_x_vals_model_ev_1)
    y_vals = value.(optimal_y_vals_model_ev_1)
    s_vals = value.(optimal_s_vals_model_ev_1)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)


    # Output stations and their attributes
    println("Deterministic Optimization 2022-23 (second stage):")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_ev_2))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1366680 rows, 653400 columns and 2608200 nonzeros
Model fingerprint: 0x5445a66e
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [3e-01, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-09, 4e+04]
Presolve removed 1366680 rows and 653400 columns
Presolve time: 0.67s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6178291e+07   0.000000e+00   0.000000e+00      1s

Solved 

In [8]:
# Get Stage 2 EV Variable Optimal Values and Objective Value
obj_ev_2022_23 = objective_value(model_ev_2)
z_ev_2022_23 = total_ontime_officers
z_prime_ev_2022_23 = total_late_officers
u_ev_2022_23 = total_late_crimes

2518.0

## Deterministic Optimization (test set on 2024 data)

In [9]:
# Initialize model
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids if haskey(node_id_to_idx, node))
model_ev_test = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset#collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  # Limit crime types

# Variables
@variable(model_ev_test, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_ev_test, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_ev_test, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_ev_test, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_model_ev_1[j])  # Coverage radius

# Objective Function
@objective(model_ev_test, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_ev_test, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] <= 300 * optimal_x_vals_model_ev_1[j])  # Don't assign officers unless station exists
@constraint(model_ev_test, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] >= 40 - 300 * (1 - optimal_x_vals_model_ev_1[j]))  # At least 40 officers if station exists
@constraint(model_ev_test, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] >= 100 * optimal_s_vals_model_ev_1[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_ev_test, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] <= 99 * (1 - optimal_s_vals_model_ev_1[j]) + 300 * optimal_s_vals_model_ev_1[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_ev_test, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_model_ev_1[j, i]
)

# Quick response constraint
@constraint(model_ev_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - optimal_a_vals_model_ev_1[j, i])
)

# Late response constraint
@constraint(model_ev_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * optimal_a_vals_model_ev_1[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_ev_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic2024[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_ev_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_ev_test, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_model_ev_1[j]
)

# Non-negativity constraints
@constraint(model_ev_test, [j in limited_candidateset], optimal_y_vals_model_ev_1[j] >= 0)
@constraint(model_ev_test, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_ev_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_ev_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_ev_test, "TimeLimit", 300)  # Time limit for solving
set_optimizer_attribute(model_ev_test, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_ev_test, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_ev_test)

# Results
if termination_status(model_ev_test) == MOI.OPTIMAL || termination_status(model_ev_test) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(optimal_x_vals_model_ev_1)
    y_vals = value.(optimal_y_vals_model_ev_1)
    s_vals = value.(optimal_s_vals_model_ev_1)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("EV 2024:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_ev_test))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 710460 rows, 326700 columns and 1304100 nonzeros
Model fingerprint: 0x300dc57a
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [5e-01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-09, 4e+04]
Presolve removed 710460 rows and 326700 columns
Presolve time: 0.47s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9269494e+07   0.000000e+00   0.000000e+00      1s

Solved in

In [10]:
# Get EV Test Variable Optimal Values and Objective Value
obj_ev_2024 = objective_value(model_ev_test)
z_ev_2024 = total_ontime_officers
z_prime_ev_2024 = total_late_officers
u_ev_2024 = total_late_crimes

513.0

## Adaptive Optimization

In [40]:
# Initialize model
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids if haskey(node_id_to_idx, node))
model_a_1 = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2 #4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 1 Variables
@variable(model_a_1, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_a_1, y[j in limited_candidateset] >= 0)  # Officers assigned to station j in scenario s
@variable(model_a_1, s_var[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_a_1, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if node i is within radius of station j

# Stage 2 Variables
@variable(model_a_1, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_a_1, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_a_1, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_a_1, D[j in limited_candidateset], 1000 + 1000 * s_var[j])  # Coverage radius

# Objective Function
@objective(model_a_1, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_a_1, sum(x[j] for j in limited_candidateset) == 3)  # Facility constraint: exactly 3 stations
@constraint(model_a_1, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers per scenario
@constraint(model_a_1, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_a_1, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_a_1, [j in limited_candidateset], y[j] >= 100 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_1, [j in limited_candidateset], y[j] <= 99 * (1 - s_var[j]) + 300 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_1, sum(s_var[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_a_1, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_a_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_a_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_1, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_1, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_a_1, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_a_1, [j in limited_candidateset], x[j] >= 0)
@constraint(model_a_1, [j in limited_candidateset], s_var[j] >= 0)
@constraint(model_a_1, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_a_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_a_1, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_1, "TimeLimit", 600)  # Increased time limit for stochastic optimization
set_optimizer_attribute(model_a_1, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_1, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_1)

# Results
if termination_status(model_a_1) == MOI.OPTIMAL || termination_status(model_a_1) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s_var)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)

    
    # Output stations and their attributes
    println("SA 2022-23:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_1))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1366683 rows, 707580 columns and 3365100 nonzeros
Model fingerprint: 0x60f3c00f
Variable types: 653460 continuous, 54120 integer (54120 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [3e-01, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-09, 4e+04]
Presolve removed 1241171 rows and 455489 columns
Presolve time: 0.76s
Presolved: 125512 rows, 252091 columns, 714452 nonzeros
Variable types: 237749 continuous, 14342 integer (14342

In [41]:
# Stage 1 Variables SA
optimal_x_vals_a = value.(x)
optimal_y_vals_a = value.(y)
optimal_s_vals_a = value.(s_var)
optimal_a_vals_a = value.(a);

In [42]:
# Get Stage 2 Variable Optimal Values and Objective Value SA
obj_a_2022_23 = objective_value(model_a_1)
z_a_2022_23 = total_ontime_officers
z_prime_a_2022_23 = total_late_officers
u_a_2022_23 = total_late_crimes

1169.0

## Adaptive Optimization (test set)

In [43]:
# Initialize model
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids if haskey(node_id_to_idx, node))
model_a_test = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  # Limit crime types

# Variables
@variable(model_a_test, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_a_test, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_a_test, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_a_test, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_a[j])  # Coverage radius

# Objective Function
@objective(model_a_test, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_a_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 300 * optimal_x_vals_a[j])  # Don't assign officers unless station exists
@constraint(model_a_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 40 - 300 * (1 - optimal_x_vals_a[j]))  # At least 40 officers if station exists
@constraint(model_a_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 100 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 99 * (1 - optimal_s_vals_a[j]) + 300 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_test, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_a[j, i]
)

# Quick response constraint
@constraint(model_a_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - optimal_a_vals_a[j, i])
)

# Late response constraint
@constraint(model_a_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * optimal_a_vals_a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic2024[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_a_test, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_a[j]
)

# Non-negativity constraints
@constraint(model_a_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 0)
@constraint(model_a_test, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_a_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_a_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_test, "TimeLimit", 300)  # Time limit for solving
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_test, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_test)

# Results
if termination_status(model_a_test) == MOI.OPTIMAL || termination_status(model_a_test) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(optimal_x_vals_a)
    y_vals = value.(optimal_y_vals_a)
    s_vals = value.(optimal_s_vals_a)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("SA 2024:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_test))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 710460 rows, 326700 columns and 1304100 nonzeros
Model fingerprint: 0x0b4b3842
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [5e-01, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-09, 4e+04]
Presolve removed 710460 rows and 326700 columns
Presolve time: 0.30s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6070740e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.50 seconds (0.40 work units)
Optimal objective  1.

In [15]:
# Get SA Test Variable Optimal Values and Objective Value
obj_a_2024 = objective_value(model_a_test)
z_a_2024 = total_ontime_officers
z_prime_a_2024 = total_late_officers
u_a_2024 = total_late_crimes

234.0

## Baseline 2022-23 (3 nodes with most crime)

In [16]:
# Initialize model only using nodes with top 3 crime count
candidate_nodes_top_3_crime = sort(candidate_nodes, :crime_count, rev=true)[1:3, :];
candidate_node_ids_top_3_crime = Set(candidate_nodes_top_3_crime.node_id)
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids_top_3_crime if haskey(node_id_to_idx, node))
model_a_1_baseline_top3_crime = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2#4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 1 Variables
@variable(model_a_1_baseline_top3_crime, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_a_1_baseline_top3_crime, y[j in limited_candidateset] >= 0)  # Officers assigned to station j in scenario s
@variable(model_a_1_baseline_top3_crime, s_var[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_a_1_baseline_top3_crime, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if node i is within radius of station j

# Stage 2 Variables
@variable(model_a_1_baseline_top3_crime, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_a_1_baseline_top3_crime, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_a_1_baseline_top3_crime, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_a_1_baseline_top3_crime, D[j in limited_candidateset], 1000 + 1000 * s_var[j])  # Coverage radius

# Objective Function
@objective(model_a_1_baseline_top3_crime, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_a_1_baseline_top3_crime, sum(x[j] for j in limited_candidateset) == 3)  # Facility constraint: exactly 3 stations
@constraint(model_a_1_baseline_top3_crime, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers per scenario
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], y[j] >= 100 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], y[j] <= 99 * (1 - s_var[j]) + 300 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_1_baseline_top3_crime, sum(s_var[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_a_1_baseline_top3_crime, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_1_baseline_top3_crime, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_1_baseline_top3_crime, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], x[j] >= 0)
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset], s_var[j] >= 0)
@constraint(model_a_1_baseline_top3_crime, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_a_1_baseline_top3_crime, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_1_baseline_top3_crime, "TimeLimit", 600)  # Increased time limit for stochastic optimization
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_1_baseline_top3_crime, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_1_baseline_top3_crime)

# Results
if termination_status(model_a_1_baseline_top3_crime) == MOI.OPTIMAL || termination_status(model_a_1_baseline_top3_crime) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s_var)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)

    # Output stations and their attributes
    println("SA 2022-23 Baseline Top 3 Crime Nodes:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_1_baseline_top3_crime))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 83727 rows, 40509 columns and 183645 nonzeros
Model fingerprint: 0x5b35c966
Variable types: 37803 continuous, 2706 integer (2706 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [2e+00, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-07, 4e+04]
Presolve removed 79059 rows and 36587 columns
Presolve time: 0.38s
Presolved: 4668 rows, 3922 columns, 11272 nonzeros
Variable types: 1316 continuous, 2606 integer (494 binary)
Found heuristic solution: objective 1.196250e+08
Found heuristic solution: object

In [17]:
# Stage 1 Variables
optimal_x_vals_a = value.(x)
optimal_y_vals_a = value.(y)
optimal_s_vals_a = value.(s_var)
optimal_a_vals_a = value.(a);

In [18]:
# Get Stage 2 Variable Optimal Values
obj_top_crime_baseline_2022_23 = objective_value(model_a_1_baseline_top3_crime)
z_top_crime_baseline_2022_23 = total_ontime_officers
z_prime_top_crime_baseline_2022_23 = total_late_officers
u_top_crime_baseline_2022_23 = total_late_crimes

4169.0

## Baseline 2024 (3 nodes with most crime)

In [19]:
# Initialize model only using nodes with top 3 crime count
candidate_nodes_top_3_crime = sort(candidate_nodes, :crime_count, rev=true)[1:3, :];
candidate_node_ids_top_3_crime = Set(candidate_nodes_top_3_crime.node_id)
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids_top_3_crime if haskey(node_id_to_idx, node))
model_a_baseline_top3_crime_test = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  # Limit crime types

# Variables
@variable(model_a_baseline_top3_crime_test, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_a_baseline_top3_crime_test, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_a_baseline_top3_crime_test, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_a_baseline_top3_crime_test, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_a[j])  # Coverage radius

# Objective Function
@objective(model_a_baseline_top3_crime_test, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 300 * optimal_x_vals_a[j])  # Don't assign officers unless station exists
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 40 - 300 * (1 - optimal_x_vals_a[j]))  # At least 40 officers if station exists
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 100 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 99 * (1 - optimal_s_vals_a[j]) + 300 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_baseline_top3_crime_test, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_a[j, i]
)

# Quick response constraint
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - optimal_a_vals_a[j, i])
)

# Late response constraint
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * optimal_a_vals_a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_baseline_top3_crime_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic2024[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_baseline_top3_crime_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_a[j]
)

# Non-negativity constraints
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 0)
@constraint(model_a_baseline_top3_crime_test, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_a_baseline_top3_crime_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_baseline_top3_crime_test, "TimeLimit", 300)  # Time limit for solving
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_baseline_top3_crime_test, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_baseline_top3_crime_test)

# Results
if termination_status(model_a_baseline_top3_crime_test) == MOI.OPTIMAL || termination_status(model_a_baseline_top3_crime_test) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(optimal_x_vals_a)
    y_vals = value.(optimal_y_vals_a)
    s_vals = value.(optimal_s_vals_a)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("SA 2024:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_baseline_top3_crime_test))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43218 rows, 18900 columns and 72900 nonzeros
Model fingerprint: 0x7b90fbbb
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [3e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-06, 4e+04]
Presolve removed 43218 rows and 18900 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0535542e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds (0.02 work units)
Optimal objective  5.053554

In [20]:
# Get Stage 2 Variable Optimal Values and Objective Value
obj_top_crime_baseline_2024 = objective_value(model_a_baseline_top3_crime_test)
z_top_crime_baseline_2024 = total_ontime_officers
z_prime_top_crime_baseline_2024 = total_late_officers
u_top_crime_baseline_2024 = total_late_crimes

827.0

## Baseline 2022-23 (random)

In [21]:
# Initialize model only using random 3 nodes
Random.seed!(42)
candidate_nodes_random_3 = candidate_nodes[sample(1:nrow(candidate_nodes), 3; replace=false), :]
candidate_node_ids_random_3 = Set(candidate_nodes_random_3.node_id)
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids_random_3 if haskey(node_id_to_idx, node))


model_a_1_baseline_random = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2#4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 1 Variables
@variable(model_a_1_baseline_random, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_a_1_baseline_random, y[j in limited_candidateset] >= 0)  # Officers assigned to station j in scenario s
@variable(model_a_1_baseline_random, s_var[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_a_1_baseline_random, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if node i is within radius of station j

# Stage 2 Variables
@variable(model_a_1_baseline_random, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_a_1_baseline_random, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_a_1_baseline_random, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_a_1_baseline_random, D[j in limited_candidateset], 1000 + 1000 * s_var[j])  # Coverage radius

# Objective Function
@objective(model_a_1_baseline_random, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_a_1_baseline_random, sum(x[j] for j in limited_candidateset) == 3)  # Facility constraint: exactly 3 stations
@constraint(model_a_1_baseline_random, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers per scenario
@constraint(model_a_1_baseline_random, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_a_1_baseline_random, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_a_1_baseline_random, [j in limited_candidateset], y[j] >= 100 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_1_baseline_random, [j in limited_candidateset], y[j] <= 99 * (1 - s_var[j]) + 300 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_1_baseline_random, sum(s_var[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_a_1_baseline_random, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_a_1_baseline_random, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_a_1_baseline_random, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_1_baseline_random, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_1_baseline_random, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_a_1_baseline_random, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_a_1_baseline_random, [j in limited_candidateset], x[j] >= 0)
@constraint(model_a_1_baseline_random, [j in limited_candidateset], s_var[j] >= 0)
@constraint(model_a_1_baseline_random, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_a_1_baseline_random, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_a_1_baseline_random, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_1_baseline_random, "TimeLimit", 600)  # Increased time limit for stochastic optimization
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_1_baseline_random, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_1_baseline_random)

# Results
if termination_status(model_a_1_baseline_random) == MOI.OPTIMAL || termination_status(model_a_1_baseline_random) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s_var)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)

    # Output stations and their attributes
    println("SA 2022-23 Baseline Random:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_1_baseline_random))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 83727 rows, 40509 columns and 183645 nonzeros
Model fingerprint: 0x15fdfaac
Variable types: 37803 continuous, 2706 integer (2706 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [1e+00, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-07, 4e+04]
Presolve removed 79491 rows and 36836 columns
Presolve time: 0.75s
Presolved: 4236 rows, 3673 columns, 10366 nonzeros
Variable types: 1916 continuous, 1757 integer (529 binary)
Found heuristic solution: objective 1.712135e+08

Root relaxation: objective 1.64

In [22]:
# Stage 1 Variables
optimal_x_vals_a = value.(x)
optimal_y_vals_a = value.(y)
optimal_s_vals_a = value.(s_var)
optimal_a_vals_a = value.(a);

In [23]:
# Get Stage 2 Variable Optimal Values and Objective Value
obj_random_baseline_2022_23 = objective_value(model_a_1_baseline_random)
z_random_baseline_2022_23 = total_ontime_officers
z_prime_random_baseline_2022_23 = total_late_officers
u_random_baseline_2022_23 = total_late_crimes

6189.0

## Baseline 2024 (random)

In [24]:
# Initialize model only using random 3 nodes
Random.seed!(42)
candidate_nodes_random_3 = candidate_nodes[sample(1:nrow(candidate_nodes), 3; replace=false), :]
candidate_node_ids_random_3 = Set(candidate_nodes_random_3.node_id)
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids_random_3 if haskey(node_id_to_idx, node))
model_a_baseline_random_test = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  # Limit crime types

# Variables
@variable(model_a_baseline_random_test, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_a_baseline_random_test, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_a_baseline_random_test, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_a_baseline_random_test, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_a[j])  # Coverage radius

# Objective Function
@objective(model_a_baseline_random_test, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_a_baseline_random_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 300 * optimal_x_vals_a[j])  # Don't assign officers unless station exists
@constraint(model_a_baseline_random_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 40 - 300 * (1 - optimal_x_vals_a[j]))  # At least 40 officers if station exists
@constraint(model_a_baseline_random_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 100 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_baseline_random_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 99 * (1 - optimal_s_vals_a[j]) + 300 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_baseline_random_test, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_a[j, i]
)

# Quick response constraint
@constraint(model_a_baseline_random_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - optimal_a_vals_a[j, i])
)

# Late response constraint
@constraint(model_a_baseline_random_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * optimal_a_vals_a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_baseline_random_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic2024[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_baseline_random_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_a_baseline_random_test, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_a[j]
)

# Non-negativity constraints
@constraint(model_a_baseline_random_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 0)
@constraint(model_a_baseline_random_test, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_a_baseline_random_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_a_baseline_random_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_baseline_random_test, "TimeLimit", 300)  # Time limit for solving
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_baseline_random_test, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_baseline_random_test)

# Results
if termination_status(model_a_baseline_random_test) == MOI.OPTIMAL || termination_status(model_a_baseline_top3_crime_test) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(optimal_x_vals_a)
    y_vals = value.(optimal_y_vals_a)
    s_vals = value.(optimal_s_vals_a)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("SA 2024:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_baseline_random_test))
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43218 rows, 18900 columns and 72900 nonzeros
Model fingerprint: 0x765e6589
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [3e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-08, 4e+04]
Presolve removed 43218 rows and 18900 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0476472e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.02 work units)
Optimal objective  7.047647

In [25]:
# Get Stage 2 Variable Optimal Values and Objective Value
obj_random_baseline_2024 = objective_value(model_a_baseline_random_test)
z_random_baseline_2024 = total_ontime_officers
z_prime_random_baseline_2024 = total_late_officers
u_random_baseline_2024 = total_late_crimes

1242.0

## Baseline 2022-23 (kmeans finds 3 centroids in data)

In [45]:
# kmeans finds 3 centroids in data to use as 3 police nodes)
Random.seed!(1)
coordinates = hcat(candidate_nodes.x, candidate_nodes.y)
k = 3
result = kmeans(coordinates', k; maxiter=100, display=:none)
selected_indices = [argmin([norm(coordinates[i, :] - result.centers[:, c]) for i in 1:size(coordinates, 1)]) for c in 1:k]
candidate_nodes_centroids_3 = candidate_nodes[selected_indices, :]
candidate_node_ids_centroids_3 = Set(candidate_nodes_centroids_3.node_id)
candidateset = Set(node_id_to_idx[node] for node in candidate_node_ids_centroids_3 if haskey(node_id_to_idx, node))
model_a_1_baseline_kmeans = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2#4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 1 Variables
@variable(model_a_1_baseline_kmeans, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_a_1_baseline_kmeans, y[j in limited_candidateset] >= 0)  # Officers assigned to station j in scenario s
@variable(model_a_1_baseline_kmeans, s_var[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_a_1_baseline_kmeans, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if node i is within radius of station j

# Stage 2 Variables
@variable(model_a_1_baseline_kmeans, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_a_1_baseline_kmeans, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_a_1_baseline_kmeans, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_a_1_baseline_kmeans, D[j in limited_candidateset], 1000 + 1000 * s_var[j])  # Coverage radius

# Objective Function
@objective(model_a_1_baseline_kmeans, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_a_1_baseline_kmeans, sum(x[j] for j in limited_candidateset) == 3)  # Facility constraint: exactly 3 stations
@constraint(model_a_1_baseline_kmeans, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers per scenario
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], y[j] >= 100 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], y[j] <= 99 * (1 - s_var[j]) + 300 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_1_baseline_kmeans, sum(s_var[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_a_1_baseline_kmeans, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_1_baseline_kmeans, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_1_baseline_kmeans, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], x[j] >= 0)
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset], s_var[j] >= 0)
@constraint(model_a_1_baseline_kmeans, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_a_1_baseline_kmeans, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_1_baseline_kmeans, "TimeLimit", 600)  # Increased time limit for stochastic optimization
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_1_baseline_kmeans, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_1_baseline_kmeans)

# Results
if termination_status(model_a_1_baseline_kmeans) == MOI.OPTIMAL || termination_status(model_a_1_baseline_kmeans) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s_var)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)

    # Output stations and their attributes
    println("SA 2022-23 Baseline kMeans:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_1_baseline_kmeans))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 83727 rows, 40509 columns and 183645 nonzeros
Model fingerprint: 0xe2e343d9
Variable types: 37803 continuous, 2706 integer (2706 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [2e+00, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-06, 4e+04]
Presolve removed 81264 rows and 38492 columns
Presolve time: 2.27s
Presolved: 2463 rows, 2017 columns, 5818 nonzeros
Variable types: 1778 continuous, 239 integer (239 binary)
Found heuristic solution: objective 8.503080e+07

Root relaxation: objective 8.2962

In [46]:
# Stage 1 Variables
optimal_x_vals_a = value.(x)
optimal_y_vals_a = value.(y)
optimal_s_vals_a = value.(s_var)
optimal_a_vals_a = value.(a);

In [47]:
# Get Stage 2 Variable Optimal Values and Objective Value
obj_kmeans_baseline_2022_23 = objective_value(model_a_1_baseline_kmeans)
z_kmeans_baseline_2022_23 = total_ontime_officers
z_prime_kmeans_baseline_2022_23 = total_late_officers
u_kmeans_baseline_2022_23 = total_late_crimes

3030.0

## Baseline 2024 (kmeans finds 3 centroids in data)

In [48]:
# kmeans finds 3 centroids in data to use as 3 police nodes)
model_a_baseline_kmeans_test = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints

# Define Sets (Limited)
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset #collect(first(nodeset, min(100, length(nodeset))))  # Limit overall nodes
limited_crime_typeset = crime_typeset  # Limit crime types

# Variables
@variable(model_a_baseline_kmeans_test, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding quickly
@variable(model_a_baseline_kmeans_test, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Officers responding late
@variable(model_a_baseline_kmeans_test, u[i in limited_nodeset, c in limited_crime_typeset] >= 0)  # Late responses

# Expressions
@expression(model_a_baseline_kmeans_test, D[j in limited_candidateset], 1000 + 1000 * optimal_s_vals_a[j])  # Coverage radius

# Objective Function
@objective(model_a_baseline_kmeans_test, Min,
    sum(π_c[c] * u[i, c] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
    t_cost * sum(d_ij[i, j] * (z[j, i, c] + z_prime[j, i, c]) * 0.5 for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)  # Transportation cost
)

# Constraints
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 300 * optimal_x_vals_a[j])  # Don't assign officers unless station exists
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 40 - 300 * (1 - optimal_x_vals_a[j]))  # At least 40 officers if station exists
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 100 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset], optimal_y_vals_a[j] <= 99 * (1 - optimal_s_vals_a[j]) + 300 * optimal_s_vals_a[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_baseline_kmeans_test, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= optimal_a_vals_a[j, i]
)

# Quick response constraint
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z[j, i, c] <= 40000 * (1 - optimal_a_vals_a[j, i])
)

# Late response constraint
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset],
    z_prime[j, i, c] <= 40000 * optimal_a_vals_a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_baseline_kmeans_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z[j, i, c] for j in limited_candidateset) + r_c[c] * u[i, c] == r_c[c] * N_ic2024[i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_baseline_kmeans_test, [i in limited_nodeset, c in limited_crime_typeset],
    sum(z_prime[j, i, c] for j in limited_candidateset) == r_c[c] * u[i, c]
)

# Police Constraint
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset],
    sum(z[j, i, c] + z_prime[j, i, c] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * optimal_x_vals_a[j]
)

# Non-negativity constraints
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset], optimal_y_vals_a[j] >= 0)
@constraint(model_a_baseline_kmeans_test, [i in limited_nodeset, c in limited_crime_typeset], u[i, c] >= 0)
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z[j, i, c] >= 0)
@constraint(model_a_baseline_kmeans_test, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset], z_prime[j, i, c] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_baseline_kmeans_test, "TimeLimit", 300)  # Time limit for solving
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_baseline_kmeans_test, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_baseline_kmeans_test)

# Results
if termination_status(model_a_baseline_kmeans_test) == MOI.OPTIMAL || termination_status(model_a_baseline_kmeans_test) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract and print variable values
    x_vals = value.(optimal_x_vals_a)
    y_vals = value.(optimal_y_vals_a)
    s_vals = value.(optimal_s_vals_a)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)
    
    # Output stations and their attributes
    println("SA 2024:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    for i in limited_nodeset
        for c in limited_crime_typeset
            # Late crimes (u)
            total_late_crimes += u_vals[i, c]
            
            # On-time officers (z)
            total_ontime_officers += sum(z_vals[j, i, c] for j in limited_candidateset)
            
            # Late officers (z_prime)
            total_late_officers += sum(z_prime_vals[j, i, c] for j in limited_candidateset)
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_baseline_kmeans_test))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 43218 rows, 18900 columns and 72900 nonzeros
Model fingerprint: 0xafec652c
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [5e+00, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-07, 4e+04]
Presolve removed 43218 rows and 18900 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.6823532e+07   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.02 work units)
Optimal objective  3.682353

In [49]:
# Get Stage 2 Variable Optimal Values and Objective Value
obj_kmeans_baseline_2024 = objective_value(model_a_baseline_kmeans_test)
z_kmeans_baseline_2024 = total_ontime_officers
z_prime_kmeans_baseline_2024 = total_late_officers
u_kmeans_baseline_2024 = total_late_crimes

635.0

## 1 Station

In [ ]:
# Create the candidate set with the single node
Random.seed!(1)
node = 61317423
candidateset = Set([node_id_to_idx[node]])
model_a_1_baseline_existing_station = Model(Gurobi.Optimizer)

# Parameters
M = 1e6  # Large constant for Big-M constraints
num_scenarios = 2 #4  # Number of scenarios
scenario_weights = [1 / num_scenarios for _ in 1:num_scenarios]  # Equal weights for scenarios

# Define Sets
limited_candidateset = candidateset #collect(first(candidateset, min(20, length(candidateset))))  # Limit candidate nodes
limited_nodeset = nodeset  # All nodes
limited_crime_typeset = crime_typeset  # All crime types
scenario_set = 1:num_scenarios  # Scenarios

# Stage 1 Variables
@variable(model_a_1_baseline_existing_station, x[j in limited_candidateset], Bin)  # Binary: 1 if a station is built at node j
@variable(model_a_1_baseline_existing_station, y[j in limited_candidateset] >= 0)  # Officers assigned to station j in scenario s
@variable(model_a_1_baseline_existing_station, s_var[j in limited_candidateset], Bin)  # Binary: 1 if station j has extended radius
@variable(model_a_1_baseline_existing_station, a[j in limited_candidateset, i in limited_nodeset], Bin)  # Binary: 1 if node i is within radius of station j

# Stage 2 Variables
@variable(model_a_1_baseline_existing_station, z[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding quickly
@variable(model_a_1_baseline_existing_station, z_prime[j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Officers responding late
@variable(model_a_1_baseline_existing_station, u[i in limited_nodeset, c in limited_crime_typeset, s in scenario_set] >= 0)  # Late responses in scenario s

# Expressions
@expression(model_a_1_baseline_existing_station, D[j in limited_candidateset], 1000 + 1000 * s_var[j])  # Coverage radius

# Objective Function
@objective(model_a_1_baseline_existing_station, Min,
    sum(scenario_weights[s] * (
        sum(π_c[c] * u[i, c, s] for i in limited_nodeset, c in limited_crime_typeset) +  # Penalty for late responses
        t_cost * sum(d_ij[i, j] * (z[j, i, c, s] + z_prime[j, i, c, s]) * 0.5
                     for j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset)
    ) for s in scenario_set)
)

# Constraints
@constraint(model_a_1_baseline_existing_station, sum(x[j] for j in limited_candidateset) == 1)  # Facility constraint: exactly 3 stations
@constraint(model_a_1_baseline_existing_station, sum(y[j] for j in limited_candidateset) == total_officers)  # Total officers per scenario
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], y[j] <= 300 * x[j])  # Don't assign officers unless station exists
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], y[j] >= 40 - 300 * (1 - x[j]))  # At least 40 officers if station exists
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], y[j] >= 100 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], y[j] <= 99 * (1 - s_var[j]) + 300 * s_var[j])  # If `s[j] = 1`, `y[j] >= 100`

# Radius coverage constraint
@constraint(model_a_1_baseline_existing_station, sum(s_var[j] for j in limited_candidateset) == 2)  # At most one station can have extended radius
@constraint(model_a_1_baseline_existing_station, [i in limited_nodeset, j in limited_candidateset],
    (d_ij[i, j] - D[j]) / M <= a[j, i]
)

# Quick response constraint
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z[j, i, c, s] <= 40000 * (1 - a[j, i])
)

# Late response constraint
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    z_prime[j, i, c, s] <= 40000 * a[j, i]
)

# Crime satisfaction constraints within Radius
@constraint(model_a_1_baseline_existing_station, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z[j, i, c, s] for j in limited_candidateset) + r_c[c] * u[i, c, s] == r_c[c] * N_ics[s][i, c]
)

# Crime satisfaction constraints outside Radius
@constraint(model_a_1_baseline_existing_station, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set],
    sum(z_prime[j, i, c, s] for j in limited_candidateset) == r_c[c] * u[i, c, s]
)

# Police Capacity Constraints
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset, s in scenario_set],
    sum(z[j, i, c, s] + z_prime[j, i, c, s] for i in limited_nodeset, c in limited_crime_typeset) <= 40000 * x[j]
)

# Non-negativity constraints
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], x[j] >= 0)
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset], s_var[j] >= 0)
@constraint(model_a_1_baseline_existing_station, [i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], u[i, c, s] >= 0)
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z[j, i, c, s] >= 0)
@constraint(model_a_1_baseline_existing_station, [j in limited_candidateset, i in limited_nodeset, c in limited_crime_typeset, s in scenario_set], z_prime[j, i, c, s] >= 0)

# Solver Parameters
set_optimizer_attribute(model_a_1_baseline_existing_station, "TimeLimit", 600)  # Increased time limit for stochastic optimization
#set_optimizer_attribute(model, "MIPGap", 0.01)    # Allowable gap for suboptimal solutions
set_optimizer_attribute(model_a_1_baseline_existing_station, "OutputFlag", 1)   # Enable solver output

# Solve the Model
optimize!(model_a_1_baseline_existing_station)

# Results
if termination_status(model_a_1_baseline_existing_station) == MOI.OPTIMAL || termination_status(model_a_1_baseline_existing_station) == MOI.TIME_LIMIT
    println("Solution found (may not be optimal).")
    
    # Extract variable values
    x_vals = value.(x)
    y_vals = value.(y)
    s_vals = value.(s_var)
    z_vals = value.(z)
    z_prime_vals = value.(z_prime)
    u_vals = value.(u)

    # Output stations and their attributes
    println("SA 2022-23 Baseline kMeans:")
    println("Stations built at:")
    for j in limited_candidateset
        if x_vals[j] > 0.5
            node_id = node_idx_to_id[j]  # Map index to node ID
            println("Node ID $node_id: Officers = $(y_vals[j]), Extended Radius = $(s_vals[j])")
        end
    end

    # Output the number of late crimes (u), on-time officers (z), and late officers (z_prime)
    total_late_crimes = 0
    total_late_officers = 0
    total_ontime_officers = 0

    # Iterate through nodes, crime types, and scenarios to calculate totals
    for i in limited_nodeset
        for c in limited_crime_typeset
            for s in scenario_set
                # Late crimes (u)
                total_late_crimes += u_vals[i, c, s]
                
                # On-time officers (z)
                total_ontime_officers += sum(z_vals[j, i, c, s] for j in limited_candidateset)
                
                # Late officers (z_prime)
                total_late_officers += sum(z_prime_vals[j, i, c, s] for j in limited_candidateset)
            end
        end
    end

    # Print the total values
    println("Total Late Crimes (u): $total_late_crimes")
    println("Total Late Officers (z_prime): $total_late_officers")
    println("Total On-Time Officers (z): $total_ontime_officers")
else
    println("No solution found. Model Status: ", termination_status(model_a_1_baseline_existing_station))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-22
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1195G7 @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 38711 rows, 17103 columns and 72015 nonzeros
Model fingerprint: 0x90cf1c38
Variable types: 16201 continuous, 902 integer (902 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+04]
  Objective range  [2e+00, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-07, 4e+04]
Presolve removed 16374 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

## Summary Table 2022-23 Results

In [50]:
# List of models and their names
models = [
    (model_a_1_baseline_random, "Random Baseline"),
    (model_a_1_baseline_top3_crime, "Top 3 Nodes With Most Crime Baseline"),
    (model_a_1_baseline_kmeans, "kMeans Baseline"),
    (model_ev_2, "Deterministic Optimization"),
    (model_a_1, "Adaptive Optimization"),
]

# Objective values, realized profits, and invested startups
objective_values = [obj_random_baseline_2022_23, obj_top_crime_baseline_2022_23, obj_kmeans_baseline_2022_23, obj_ev_2022_23, obj_a_2022_23]
total_on_time_officers = [z_random_baseline_2022_23, z_top_crime_baseline_2022_23, z_kmeans_baseline_2022_23, z_ev_2022_23, z_a_2022_23]
total_late_officers = [z_prime_random_baseline_2022_23, z_prime_top_crime_baseline_2022_23, z_prime_kmeans_baseline_2022_23, z_prime_ev_2022_23, z_prime_a_2022_23]
total_late_crimes = [u_random_baseline_2022_23, u_top_crime_baseline_2022_23, u_kmeans_baseline_2022_23, u_ev_2022_23, u_a_2022_23]

using PrettyTables

# Build the DataFrame
summary_table_2022_23 = DataFrame(
    Model = [model[2] for model in models],  # Extract model names
    Objective_Values = objective_values,
    Total_Late_Crimes = total_late_crimes,
    Total_Late_Officers = total_late_officers,
    Total_On_Time_Officers = total_on_time_officers
)

# Define custom formatting functions to avoid scientific notation
formatters = (
    ft_printf("%0.2f"),   # For Model column (default string formatting)
    ft_printf("%f"), # For In_Sample_Profit column
    ft_printf("%f"), # For Realized_Profit column
    ft_printf("%f")    # For Num_Invested_Startups column
)

# Display the summary table with PrettyTables
println("2022-23 Models")
pretty_table(summary_table_2022_23, formatters = formatters)

2022-23 Models
┌──────────────────────────────────────┬──────────────────┬───────────────────┬─────────────────────┬────────────────────────┐
│                                Model │ Objective_Values │ Total_Late_Crimes │ Total_Late_Officers │ Total_On_Time_Officers │
│                               String │          Float64 │           Float64 │             Float64 │                Float64 │
├──────────────────────────────────────┼──────────────────┼───────────────────┼─────────────────────┼────────────────────────┤
│                      Random Baseline │     164231437.29 │           6189.00 │            29310.00 │               34766.00 │
│ Top 3 Nodes With Most Crime Baseline │     113432700.16 │           4169.00 │            19872.00 │               44204.00 │
│                      kMeans Baseline │      82962397.24 │           3030.00 │            14454.00 │               49622.00 │
│           Deterministic Optimization │      66178291.30 │           2518.00 │            11700

## Summary Table 2024 Results

In [51]:
# List of models and their names
models = [
    (model_a_baseline_random_test, "Random Baseline"),
    (model_a_baseline_top3_crime_test, "Top 3 Nodes With Most Crime Baseline"),
    (model_a_baseline_kmeans_test, "kMeans Baseline"),
    (model_ev_test, "Deterministic Optimization"),
    (model_a_test, "Adaptive Optimization"),
]

# Objective values, realized profits, and invested startups
objective_values = [obj_random_baseline_2024, obj_top_crime_baseline_2024, obj_kmeans_baseline_2024, obj_ev_2024, obj_a_2024]
total_on_time_officers = [z_random_baseline_2024, z_top_crime_baseline_2024, z_kmeans_baseline_2024, z_ev_2024, z_a_2024]
total_late_officers = [z_prime_random_baseline_2024, z_prime_top_crime_baseline_2024, z_prime_kmeans_baseline_2024, z_prime_ev_2024, z_prime_a_2024]
total_late_crimes = [u_random_baseline_2024, u_top_crime_baseline_2024, u_kmeans_baseline_2024, u_ev_2024, u_a_2024]

using PrettyTables

# Build the DataFrame
summary_table_2024 = DataFrame(
    Model = [model[2] for model in models],  # Extract model names
    Objective_Values = objective_values,
    Total_Late_Crimes = total_late_crimes,
    Total_Late_Officers = total_late_officers,
    Total_On_Time_Officers = total_on_time_officers
)

# Define custom formatting functions to avoid scientific notation
formatters = (
    ft_printf("%0.2f"),   # For Model column (default string formatting)
    ft_printf("%f"), # For In_Sample_Profit column
    ft_printf("%f"), # For Realized_Profit column
    ft_printf("%f")    # For Num_Invested_Startups column
)

# Display the summary table with PrettyTables
println("2024 Models")
pretty_table(summary_table_2024, formatters = formatters)

2024 Models
┌──────────────────────────────────────┬──────────────────┬───────────────────┬─────────────────────┬────────────────────────┐
│                                Model │ Objective_Values │ Total_Late_Crimes │ Total_Late_Officers │ Total_On_Time_Officers │
│                               String │          Float64 │           Float64 │             Float64 │                Float64 │
├──────────────────────────────────────┼──────────────────┼───────────────────┼─────────────────────┼────────────────────────┤
│                      Random Baseline │      70476471.87 │           1242.00 │             6114.00 │                7370.00 │
│ Top 3 Nodes With Most Crime Baseline │      50535542.04 │            827.00 │             4152.00 │                9332.00 │
│                      kMeans Baseline │      36823531.90 │            635.00 │             3138.00 │               10346.00 │
│           Deterministic Optimization │      29269493.50 │            513.00 │             2482.00